# Mining testimonial fragments of the Holocaust

**Experience domain:**

### Load the necessary libraries

In [3]:
import sys; sys.path.insert(0, '..')
import itertools

In [4]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [5]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random
import pandas as pd

### Helper functions

In [6]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [7]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [8]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [9]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [10]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [11]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [12]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [13]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

In [47]:
def update_fragments(main_node_label,mid_node_label,testimony_id,attributes):
    results = mongo.fragments.find({'label':main_node_label},{'_id':0})[0]
    mongo_id= mongo.fragments.find({'label':main_node_label},{'_id':1})[0]
    retrieved = False
    for x,element in enumerate(results['tree']['children']):
        if element['label']==mid_node_label:
            retrieved = True
            for n,leaves in enumerate(element['children']):
                if leaves['testimony_id'] == testimony_id:
                    for attr in attributes:
                        results['tree']['children'][x]['children'][n][list(attr.keys())[0]]=attr[list(attr.keys())[0]]
    if retrieved:
        replace = mongo.fragments.replace_one(mongo_id,results)
    else:
        print ('not found')
        print (main_node_label)
        print (mid_node_label)
        print (testimony_id)


1. Read the CSV file
2. Iterate through
3. Find beginning and end sentence again
4. Try to update the fragment,if not, print it out

In [53]:
input_data = "data/Input/Fragments/testimonial_fragments_old_for_reprocessing.csv"

In [63]:
df = pd.read_csv(input_data)
for row in df.iterrows():
    try:
        original_sent = row[1]['original_sentences']
        main_node = row[1]['main_leaf']
        mid_node = row[1]['mid_leaf']
        testimony_id = row[1]['testimony_id']
        indices = find_sentence_id(original_sent)
        update_fragments(main_node,mid_node,testimony_id,[{'start_sentence_index':indices[0]},{'end_sentence_index':indices[1]}])
    except:
        print ("not found")
        print (row[1])

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22to%22%5D+%5B%22a%22%5D+%5B%22young%22%5D+%5B%22kid%22%5D+%5B%22to%22%5D+%5B%22see%22%5D+%5B%22all%22%5D+%5B%22these%22%5D+%5B%22naked%22%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22never%22%5D+%5B%22even%22%5D+%5B%22saw%22%5D+%5B%22my%22%5D+%5B%22mother%22%5D+%5B%22naked%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'to a young kid to see all these naked -- I never even saw my mother naked ', 'right': '', 'complete_match': 'to a young kid to see all these naked -- I never even saw my mother naked ', 'testimony_id': 'usc_shoah_1029', 'shelfmark': ['USC Shoah Foundation 1029'], 'token_start': 8905, 'token_end': 8922}
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Because%22%5D+%5B%22I%22%5D+%5B%22could%22%5D+%5B%22never%22%5D+%5B%22see%22%5D+%5B%22anybody%22%5D+%5B%22staring%22%5D+%5B%22at%22%5D+%5B%22me%22%5D+%5B%22while%22%5D+%5B%22I%22%5D+%5B%22was%22%5D

{'left': '', 'match_word': 'So we had to stand for hours and hours in rows naked , ', 'right': '', 'complete_match': 'So we had to stand for hours and hours in rows naked , ', 'testimony_id': 'usc_shoah_20088', 'shelfmark': ['USC Shoah Foundation 20088'], 'token_start': 15944, 'token_end': 15957}
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22The%22%5D+%5B%22women%22%5D+%5B%22were%22%5D+%5B%22kept%22%5D+%5B%22naked%22%5D+%5B%22for%22%5D+%5B%22about%22%5D+%5B%22five%22%5D+%5B%22hours%22%5D+%5B%5D+%5B%22It%22%5D+%5B%22was%22%5D+%5B%22just%22%5D+%5B%22one%22%5D+%5B%22of%22%5D+%5B%22these%22%5D+%5B%22German%22%5D+%5B%22things%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'The women were kept naked for about five hours . It was just one of these German things . ', 'right': '', 'complete_match': 'The women were kept naked for about five hours . It was just one of these German things . ', 'testimony_id': 'usc_shoah

{'left': '', 'match_word': 'Everybody was half naked outside and was February , I think – the beginning of March cold . Terribly . And a lot of us got pneumonia . ', 'right': '', 'complete_match': 'Everybody was half naked outside and was February , I think – the beginning of March cold . Terribly . And a lot of us got pneumonia . ', 'testimony_id': 'irn504582', 'shelfmark': ['USHMM RG-50.030*0087'], 'token_start': 6437, 'token_end': 6465}
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22we%22%5D+%5B%22found%22%5D+%5B%22once%22%5D+%5B%22in%22%5D+%5B%22a%22%5D+%5B%22while%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22forest%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22little%22%5D+%5B%22streams%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22found%22%5D+%5B%5D%7B0%2C50%7D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22cold%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22were%22%5D+%5B%22almost%22%5D+%5B%22naked%22%5D+%5B%5D+%5B%22No%22%5D+%5B%22clothes%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=n

{'left': '', 'match_word': 'As we were marching down in Hamburg , we encountered a group of French prisoners who were their prisoners of war . But they were still getting packages through the Red Cross . But when they saw us half naked , starved , women , they started to take off their jackets ', 'right': '', 'complete_match': 'As we were marching down in Hamburg , we encountered a group of French prisoners who were their prisoners of war . But they were still getting packages through the Red Cross . But when they saw us half naked , starved , women , they started to take off their jackets ', 'testimony_id': 'HVT-34', 'shelfmark': ['Fortunoff Archive HVT-34'], 'token_start': 11331, 'token_end': 11383}
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%22undress%22%5D+%5B%22and%22%5D+%5B%22march%22%5D+%5B%22naked%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We had to undress

{'left': '', 'match_word': 'They was standing there talking to one another , laughing their head off that we were all standing there in – with nothing on helplessly . ', 'right': '', 'complete_match': 'They was standing there talking to one another , laughing their head off that we were all standing there in – with nothing on helplessly . ', 'testimony_id': 'irn504539', 'shelfmark': ['USHMM RG-50.030*0035'], 'token_start': 7412, 'token_end': 7438}
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22were%22%5D+%5B%22told%22%5D+%5B%22to%22%5D+%5B%22get%22%5D+%5B%22completely%22%5D+%5B%22undressed%22%5D+%5B%22naked%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22guards%22%5D+%5B%22standing%22%5D+%5B%22around%22%5D+%5B%22and%22%5D+%5B%22around%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22know%22%5D+%5B%22when%22%5D+%5B%22I%22%5D+%5B%22think%22%5D+%5B%22of%22%5D+%5B%22that%22%5D+%5B%22moment%22%5D+%5B%22I%22%5D+%5B%22still%22%5D+%5B%22remember%22%5D+%5B%22how%22%5D+%5B%22I%22%5D+%5B%22d

...

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22It%22%5D+%5B%22was%22%5D+%5B%22such%22%5D+%5B%22a%22%5D+%5B%22shame%22%5D+%5B%22to%22%5D+%5B%22be%22%5D+%5B%22Jewish%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'It was such a shame to be Jewish . ', 'right': '', 'complete_match': 'It was such a shame to be Jewish . ', 'testimony_id': 'usc_shoah_19939', 'shelfmark': ['USC Shoah Foundation 19939'], 'token_start': 23856, 'token_end': 23865}
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22was%22%5D+%5B%22always%22%5D+%5B%22ashamed%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22ashamed%22%5D+%5B%22to%22%5D+%5B%22be%22%5D+%5B%22Jewish%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I was always ashamed . I was ashamed to be Jewish . ', 'right': '', 'complete_match': 'I was always ashamed . I was ashamed to be Jewish . ', 't

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22felt%22%5D+%5B%22somehow%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22remarks%22%5D+%5B%22from%22%5D+%5B%22the%22%5D+%5B%22teachers%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22remarks%22%5D+%5B%22from%22%5D+%5B%22the%22%5D+%5B%22former%22%5D+%5B%22neighbors%22%5D+%5B%5D+%5B%22degrading%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And we , we felt somehow , the remarks from the teachers , the remarks from the former neighbors , degrading . ', 'right': '', 'complete_match': 'And we , we felt somehow , the remarks from the teachers , the remarks from the former neighbors , degrading . ', 'testimony_id': 'usc_shoah_747', 'shelfmark': ['USC Shoah Foundation 747'], 'token_start': 5778, 'token_end': 5800}
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22they%22%5D+%5B%22tortured%22%5D+%5B%22her%22%5D+%5B%22and%22%5D+%5

{'left': '', 'match_word': "I did n't find out what I could do to save them . I will always have those guilt feelings . ", 'right': '', 'complete_match': "I did n't find out what I could do to save them . I will always have those guilt feelings . ", 'testimony_id': 'usc_shoah_7600', 'shelfmark': ['USC Shoah Foundation 7600'], 'token_start': 36022, 'token_end': 36043}
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22still%22%5D+%5B%22I%22%5D+%5B%22feel%22%5D+%5B%22that%22%5D+%5B%22I%22%5D+%5B%22should%22%5D+%5B%22have%22%5D+%5B%22done%22%5D+%5B%22something%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'still I feel that I should have done something ', 'right': '', 'complete_match': 'still I feel that I should have done something ', 'testimony_id': 'irn508626', 'shelfmark': ['USHMM RG-50.462*0005'], 'token_start': 30606, 'token_end': 30615}
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Certain%22%5D+%

{'left': '', 'match_word': "I blame myself for being guilty possibly of my father 's death . ", 'right': '', 'complete_match': "I blame myself for being guilty possibly of my father 's death . ", 'testimony_id': 'usc_shoah_16345', 'shelfmark': ['USC Shoah Foundation 16345'], 'token_start': 12627, 'token_end': 12640}
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22had%22%5D+%5B%22tremendous%22%5D+%5B%22guilt%22%5D+%5B%22feelings%22%5D+%5B%22that%22%5D+%5B%22I%22%5D+%5B%22left%22%5D+%5B%22without%22%5D+%5B%22saying%22%5D+%5B%22goodbye%22%5D+%5B%22to%22%5D+%5B%22my%22%5D+%5B%22father%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I had tremendous guilt feelings that I left without saying goodbye to my father . ', 'right': '', 'complete_match': 'I had tremendous guilt feelings that I left without saying goodbye to my father . ', 'testimony_id': 'usc_shoah_3022', 'shelfmark': ['USC Shoah Foundation 3

{'left': '', 'match_word': 'Seeing small children that are innocent being thrown into the fire . If I am innocent -- if I am guilty , fine . ', 'right': '', 'complete_match': 'Seeing small children that are innocent being thrown into the fire . If I am innocent -- if I am guilty , fine . ', 'testimony_id': 'irn507287', 'shelfmark': ['USHMM RG-50.030*0397'], 'token_start': 9289, 'token_end': 9313}


In [62]:
row[1]['original_sentences']

'But, and they laughed and laughed that they left them alive but completely naked and took away their clothes.'

### Add the main node

In [109]:
main_node = ""
#delete_main_node(main_node)
add_main_node(main_node)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

In [110]:
query = ''

In [ ]:
result = topic_concordancer.main(query,window=50,topicn=25)

### Print the key topics

In [ ]:
for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

### Analyze documents

In [ ]:
i=0
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

## Testimonial fragments

### 1.  

In [111]:
lemmas = []

In [112]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="kill"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="kill"])


In [113]:
domain_term = ""

In [114]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [115]:
fragment_1 = {}
fragment_1['original_sentence'] = ""
fragment_1['label']=""
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%223%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22Then%22%5D+%5B%22they%22%5D+%5B%22said%22%5D+%5B%5D+%5B%22you%22%5D+%5B%5D+%5B%22out%22%5D+%5B%22of%22%5D+%5B%22luck%22%5D+%5B%22because%22%5D+%5B%22we%22%5D+%5B%5D+%5B%22going%22%5D+%5B%22to%22%5D+%5B%22kill%22%5D+%5B%22you%22%5D+%5B%22all%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%224%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22We%22%5D+%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%224%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22were%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "men . [ PAUSES FOR 3 SECONDS ] Then they said , you 're out of luck because we 're going to kill you all . [ PAUSES FOR 4 SECONDS ] We [ PAUSES FOR 4 SECONDS ] were numb . ", 'right': '', 'complete_match': "men . [ 

In [116]:
fragment_2 = {}
fragment_2['original_sentence'] = ""
fragment_2['label']=""
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22To%22%5D+%5B%22see%22%5D+%5B%22another%22%5D+%5B%5D+%5B%22Jews%22%5D+%5B%22come%22%5D+%5B%22and%22%5D+%5B%22be%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22another%22%5D+%5B%5D+%5B%22be%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22That%22%5D+%5B%22was%22%5D+%5B%22not%22%5D+%5B%5D%7B0%2C50%7D+%5B%22that%22%5D+%5B%22we%22%5D+%5B%22wanted%22%5D+%5B%22to%22%5D+%5B%22see%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%5D+%5B%22Then%22%5D+%5B%22after%22%5D+%5B%22awhile%22%5D+%5B%22you%22%5D+%5B%22become%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%22that%22%5D+%5B%22you%22%5D+%5B%22just%22%5D+%5B%5D%7B0%2C50%7D+%5B%22you%22%5D+%5B%22do%22%5D+%5B%5D+%5B%22react%22%5D+%5B%5D+%5B%22you%22%5D+%5B%5D%7B0%2C50%7D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "To see another 10,000 Jews come and be killed , ano

In [117]:
fragment_3 = {}
fragment_3['original_sentence'] = ""
fragment_3['label']=""
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%5D+%5B%22Cathy%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22every%22%5D+%5B%2210%22%5D+%5B%22people%22%5D+%5B%22get%22%5D+%5B%22killed%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And we were so numb , [ ? Cathy ? ] every 10 people get killed . ', 'right': '', 'complete_match': 'And we were so numb , [ ? Cathy ? ] every 10 people get killed . ', 'testimony_id': 'usc_shoah_25287', 'shelfmark': ['USC 25287'], 'token_start': 16040, 'token_end': 16057}


In [118]:
fragment_4 = {}
fragment_4['original_sentence'] = ""
fragment_4['label']= ""
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22from%22%5D+%5B%22people%22%5D+%5B%22being%22%5D+%5B%22wounded%22%5D+%5B%5D+%5B%22from%22%5D+%5B%22people%22%5D+%5B%22being%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22from%22%5D+%5B%22cries%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22finally%22%5D+%5B%5D+%5B%22everything%22%5D+%5B%22subsided%22%5D+%5B%5D+%5B%22But%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22alive%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22even%22%5D+%5B%22think%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "from people being wounded , from people being killed , from cries . And finally , everything subsided . But I was alive . I did n't -- I did n't even think . I was numb . ", 'right': '', 'complete_match': "from people being wounded , from people being killed , from cries . And finally , ev

In [ ]:
fragment_5 = {}
fragment_5['original_sentence'] = ""
fragment_5['label']= ""
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

In [119]:
add_testimonial_fragments(fragments)